In [ ]:
import json
with open("config.json", "rt") as f:
    config_obj = json.load(f)
print(config_obj)
globals().update(config_obj)

{'dataset_path': '<path to your chb-mit directory> e.g ', 'clean_data_path': '<path to store clean & aligned dataset> e.g ./data_clean', 'label_output_path': '<path to store generated TOML files> e.g ./ref_labels', 'ignore_lst': []}


In [3]:
from datetime import datetime, timedelta

# clean_data_path = "./swecethz-clean/"
plot_args_path = timeline_info_path; os.makedirs(plot_args_path, exist_ok=True)
# TODO Combined with command-line arguments
with open(os.path.join("..", "global_config.json"), "rt") as f:
    config_obj = json.load(f)
# globals().update(config_obj)
dt_fmt = config_obj.datetime_fmt

fake_start_dt = datetime.strptime(fake_start_datetime, dt_fmt)

import os
import json

with os.scandir(clean_data_path) as entries:
    for entry in entries:
        if entry.is_dir():
            # subdir_path = entry.path

            result_obj = {"record_lst": [], "seizure_lst": [], "unused_rec_idx_lst": []}
            with open(os.path.join(entry.path, entry.name+"_info.json"), "rt") as finfo:
                info = json.load(finfo)
                for sbeg, send in zip(info["seizure_begin"], info["seizure_end"]):
                    result_obj["seizure_lst"].append({
                        "span": [(fake_start_dt+timedelta(seconds=sbeg)).strftime(dt_fmt), 
                                 (fake_start_dt+timedelta(seconds=send)).strftime(dt_fmt)], 
                        "info": f"Onset {fake_start_dt+timedelta(seconds=sbeg)}, last {send-sbeg}s"
                    })
                fs = info["fs"]
            with open(os.path.join(entry.path, entry.name+"_data.json"), "rt") as fdata:
                matsinfo = json.load(fdata)
                cur_dt = fake_start_dt
                for k, mat in enumerate(matsinfo):
                    new_dt = cur_dt+timedelta(seconds=(mat[1]["__shape__"][1]/fs))
                    result_obj["record_lst"].append({
                        "file": mat[0], 
                        "span": [cur_dt.strftime(dt_fmt), 
                                 new_dt.strftime(dt_fmt)], 
                        "info": f"{mat[0]} records {cur_dt} ~ {new_dt} \r\n of shape {mat[1]['__shape__']} "
                                    f"{cur_dt} ~ {new_dt}"
                    })
                    if mat[0] in ignore_lst:
                        result_obj["unused_rec_idx_lst"].append(k)
                    cur_dt = new_dt

            with open(os.path.join(plot_args_path, entry.name+".json"), "wt") as f:
                json.dump(result_obj, f, indent=2)

